In [1]:
import re
import time
import pandas as pd
import numpy as np
import nltk
from nltk import *
import json

import random
random.seed(10)

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

# Ejemplo con Stack de RNNs
from keras.layers import Embedding, SimpleRNN
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from keras.models import Sequential, load_model
from keras.losses import SparseCategoricalCrossentropy

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score #son metricas - area debajo de la curva roc

from sklearn.model_selection import train_test_split




### Lectura del archivo de tweets ingestados y etiquetados

In [2]:
corpus_train_esA = pd.read_csv('tweets_search_etiquetas.csv',delimiter='\t',encoding='latin-1', dtype = {'id_str': str})
corpus_train_esA["id_str"] = corpus_train_esA["id_str"].astype(str).replace(to_replace="[.]", value = "", regex=True)
corpus_train_esA["id_str"] = corpus_train_esA["id_str"].astype(str).replace(to_replace="E\+\d{3}", value = "", regex=True)

corpus_train_esA.head(10)

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention
253,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,La app de bancomer si me ha funcionado en huawei,1,1,1,0
254,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,"Los free buds de huawei son buenos. Baratos, d...",1,1,1,0
255,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,La carga rapida del huawei nova 9 es muy buena,1,1,1,0
256,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,El servicio de correo de huawei se puede sincr...,1,1,1,0
257,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,tengo whatsapp sin problemas en mi huawei p20 ...,1,1,1,0
258,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,he jugado pug mobile en mi nova 9 sin ningun p...,1,1,1,0
259,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,la camara del huawei mate pro tiene un buen zo...,1,1,1,0
260,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,pensé que por tener un huawei estaria limitado...,1,1,1,0
261,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,igualmente utilizo el servicio de nube de huwe...,1,1,1,0
262,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,En el servicio de tienda de huawei tambien pue...,1,1,0,1


### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links 
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)


In [3]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):    
    file[file.columns[3]] = [clean_text(i) for i in file[file.columns[3]]]    

    sentiment_code_dict = {"000":0, "010":1, "001":2, "110":3, "101":4}
    file["sentiment_code"] = file["sentiment"].map(str)  + file["funcionality"].map(str)  + file["client_attention"].map(str) 
    file["sentiment_code"] = file["sentiment_code"].apply(lambda x: sentiment_code_dict[x])

    file.to_csv(namefile, sep=';', encoding='latin-1', index=False)
    return file
    
def clean_text(text):
    text = text.lower()   
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", "", text)
    text=re.sub(pattern_URL, "", text)
    
    text= remove_emoji(text)
    text = re.sub("(\d+)|(rt)|(RT)", "", text)
    text = re.sub("#\w+", "", text)
    text= remove_stopwords(text)
    
    # text=re.sub("\d+", " ", text)
    
    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):    
    stopwords=set(nltk.corpus.stopwords.words("spanish"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs                               
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "]+", flags=re.UNICODE)   
    text = emoji_pattern.sub(r'', text) # no emoji

    return text

In [4]:
corpus_train_esA = procesar(corpus_train_esA, "tweets_search_etiquetas_clean.csv")
corpus_train_esA.tail()
#corpus_train_esA = pd.read_csv("tweets_search_etiquetas_clean.csv", sep = ";", encoding = "latin-1")
#corpus_train_esA.head()

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention,sentiment_code
308,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,huawei p lite repente apaga malo,1,0,1,0,1
309,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,camara huawei funciona bien noche,1,0,1,0,1
310,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,huawei calienta demasiado,1,0,1,0,1
311,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,colores fotos huawei estan saturados malos,1,0,1,0,1
312,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,contento huawei colores estan saturados,1,0,1,0,1


### Particionamiento de los datos en conjunto de entrenamiento y conjunto de pruebas

In [5]:
corpus_train_esA, corpus_dev_esA = train_test_split(corpus_train_esA, test_size=0.3)

corpus_dev_esA.to_csv("tweets_search_etiquetas_clean_test.csv", sep = ";", encoding = "latin-1", index = False)
corpus_dev_esA = pd.read_csv("tweets_search_etiquetas_clean_test.csv", sep = ";", encoding = "latin-1")
corpus_dev_esA.tail()

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention,sentiment_code
89,153685582485693,Tue Jun 14 23:39:16 +0000 2022,fcoquijano,paila comprar celular últimos modelos huawei s...,208,0,0,0,0
90,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,cayó celular si aguantó huawei p lite,1,1,1,0,3
91,153674623721867,Tue Jun 14 16:23:48 +0000 2022,arielmcorg,voy comprar iphone pantalla huawei prime,4794,1,1,0,3
92,153653000000000,Mon Jun 13 19:06:34 +0000 2022,Juankchs,jefes firman agenda digital cumbre américas a...,311,0,1,0,1
93,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,bocina huawei buena graves suenan bien,1,1,1,0,3


In [6]:
train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[3]].fillna(' ')
y_train_hsA = corpus_train_esA[[corpus_train_esA.columns[5], corpus_train_esA.columns[6], corpus_train_esA.columns[7]]]
#y_train_hsA = corpus_train_esA[corpus_train_esA.columns[8]]
y_train_hsA = y_train_hsA.values

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[3]].fillna(' ')
y_test_hsA = corpus_dev_esA[[corpus_dev_esA.columns[5], corpus_dev_esA.columns[6], corpus_dev_esA.columns[7]]]
#y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[8]]
y_test_hsA = y_test_hsA.values

In [9]:
print(y_train_hsA.shape)

(227, 3)


In [7]:
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(3,5),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=3,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

dump(cvectorizer, 'count_vectorizer.joblib')


(219, 73)
(94, 73)


['count_vectorizer.joblib']

In [16]:
print(len(X_train_cvectorized[0]))
print(X_test_cvectorized.shape)

48
(82, 48)


In [53]:
#X_train_cvectorized
y_train_hsA

array([3, 0, 0, 0, 1, 3, 0, 1, 1, 3, 1, 0, 0, 0, 3, 0, 0, 1, 1, 3, 3, 4,
       1, 0, 1, 3, 0, 2, 3, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 4, 0, 0, 3,
       0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 2,
       0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 1, 2, 1, 3, 1, 3, 0, 0, 0, 0, 2, 3,
       0, 0, 0, 3, 3, 1, 0, 1, 0, 3, 2, 0, 1, 3, 1, 1, 0, 0, 0, 3, 2, 0],
      dtype=int64)

In [17]:
cvectorizer.vocabulary_

{'clientes chiste huawei': 3,
 'chiste huawei primero': 0,
 'huawei primero ofrecen': 20,
 'primero ofrecen cosa': 30,
 'ofrecen cosa después': 26,
 'cosa después echan': 6,
 'después echan atrás': 11,
 'echan atrás nota': 16,
 'clientes chiste huawei primero': 4,
 'chiste huawei primero ofrecen': 1,
 'huawei primero ofrecen cosa': 21,
 'primero ofrecen cosa después': 31,
 'ofrecen cosa después echan': 27,
 'cosa después echan atrás': 7,
 'después echan atrás nota': 12,
 'clientes chiste huawei primero ofrecen': 5,
 'chiste huawei primero ofrecen cosa': 2,
 'huawei primero ofrecen cosa después': 22,
 'primero ofrecen cosa después echan': 32,
 'ofrecen cosa después echan atrás': 28,
 'cosa después echan atrás nota': 8,
 'van dos semanas': 45,
 'dos semanas sigo': 13,
 'semanas sigo esperando': 39,
 'sigo esperando respuesta': 42,
 'esperando respuesta huawei': 17,
 'respuesta huawei prometen': 36,
 'huawei prometen cupones': 23,
 'prometen cupones pides': 33,
 'cupones pides ignoran': 9

In [35]:
with open("cvectorizer_vocabulary_.json", mode = "w", encoding = "latin-1", ) as f:
    f.write(json.dumps(cvectorizer.vocabulary_, indent = 4, ensure_ascii = False))


In [8]:
max_features =  10000 #len(cvectorizer.vocabulary_) or X_train_cvectorized.shape[1] #10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = X_test_cvectorized.shape[1] #1775         # longitud máxima de cada secuencia 
batch_size = 32


In [9]:

model = Sequential()
# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
model.add(Embedding(input_dim = max_features,  output_dim=32, input_length = maxlen)) # output_dim = 32
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))

model.add(Dense(3, activation='softmax'))

model.summary()

#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.compile(optimizer='adam', loss = "categorical_crossentropy", metrics=['accuracy'])


tic = time.time()
history_stackRNN = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=15,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 73, 32)            320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 73, 32)            2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 324,259
Trainable params: 324,259
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
6/6 - 10s - loss: 0.9145 - accuracy: 0.5257 - val_loss: 0.9653 - val_accuracy: 0.1818 - 10s/epoch - 2s/step
Epoch 2/15
6/6 - 1s - loss: 0.8844 - accuracy: 0.2057 - val_los

In [12]:
embedding_dim = batch_size # 50 #batch_size
#maxlen = 15

model = Sequential()
model.add(Embedding(input_dim = max_features, 
                           output_dim = embedding_dim, 
                           input_length = maxlen))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train_cvectorized, y_train_hsA,
                    epochs=15,
                    verbose=2,
                    validation_data=(X_test_cvectorized, y_test_hsA),
                    batch_size=batch_size)
                    
loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 73, 32)            320000    
                                                                 
 flatten_2 (Flatten)         (None, 2336)              0         
                                                                 
 dense_5 (Dense)             (None, 15)                35055     
                                                                 
 dense_6 (Dense)             (None, 3)                 48        
                                                                 
Total params: 355,103
Trainable params: 355,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
7/7 - 2s - loss: 1.0319 - accuracy: 0.1689 - val_loss: 0.9180 - val_accuracy: 0.2447 - 2s/epoch - 314ms/step
Epoch 2/15
7/7 - 0s - loss: 1.0182 - accuracy: 0.1781 - val_

In [28]:

lstm_out = 98

model = Sequential()
model.add(Embedding(input_dim = max_features, output_dim = embedding_dim, input_length = maxlen))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

batch_size = 32
model.fit(X_train_cvectorized, y_train_hsA, epochs = 10, batch_size=batch_size, verbose = 2)

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 48, 32)            320000    
                                                                 
 lstm_1 (LSTM)               (None, 98)                51352     
                                                                 
 dense_11 (Dense)            (None, 3)                 297       
                                                                 
Total params: 371,649
Trainable params: 371,649
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
6/6 - 7s - loss: 0.8886 - accuracy: 0.2670 - 7s/epoch - 1s/step
Epoch 2/10
6/6 - 1s - loss: 0.8215 - accuracy: 0.1832 - 815ms/epoch - 136ms/step
Epoch 3/10
6/6 - 1s - loss: 0.7730 - accuracy: 0.1832 - 836ms/epoch - 139ms/step
Epoch 4/10
6/6 - 1s - loss: 0.7677 - accuracy: 0.1832 - 764ms/epoch - 

In [13]:
model.save('best_keras_network_model.h5')

In [11]:
input_list = list(map(lambda x: str(x), y_train_hsA.round()))
c = Counter( input_list )

print( c.items() )

dict_items([('[0 0 0]', 105), ('[0 1 0]', 41), ('[1 1 0]', 45), ('[0 0 1]', 14), ('[1 0 1]', 1)])


In [14]:
from collections import Counter

input_list =  list(map(lambda x: str(x), y_train_hsA.round()))
input_list =  list(map(lambda x: str(x), y_test_hsA.round()))
input_list =  list(map(lambda x: str(x), testPredict_stackRNN.round()))
c = Counter( input_list )

print( c.items() )



dict_items([('[0. 1. 0.]', 89), ('[0. 0. 1.]', 4), ('[0. 0. 0.]', 1)])


In [33]:
#testPredict_stackRNN.round()
list(map(lambda x: str(x), testPredict_stackRNN.round()))

['[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0

In [20]:
corpus_dev_esA["predict_test"] = list(map(lambda x: str(x), testPredict_stackRNN.round()))

corpus_dev_esA.to_csv("tweets_search_etiquetas_clean_test_predict.csv", sep="\t", encoding = "latin-1")

In [21]:
list(map(lambda x: str(x), testPredict_stackRNN.round()))

['[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 0. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0

In [7]:
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd

app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
"""
df = pd.DataFrame({
    "Ambito": ["funcionamiento", "atencion_cliente", "atencion_cliente", "funcionamiento", "funcionamiento", "funcionamiento"],
    "Conteo": [4, 1, 2, 2, 4, 5],
    "Sentimiento": ["Positivo", "Negativo", "Negativo", "Negativo", "Negativo", "Positivo"]
})
"""


app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Analisis de sentimiento
    '''),

    dcc.Graph(
        id='live-update-graph'#,
        #figure=fig
    ),
    dcc.Interval(
            id='interval-component',
            interval=8*1000, # in milliseconds
            n_intervals=0
    )
])


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph_live(n):

    df = pd.read_csv("stream_tweet_groupby.csv", sep = ",", encoding= "latin-1", dtype = {'sentiment_code': str})

    df["sentiment_bin"] = df["sentiment_code"].apply(lambda x: str(x[0]))
    sentiment_dict = {"0": "negativo", "1": "positivo"}
    df["sentiment_bin"] = df["sentiment_bin"].apply(lambda x: sentiment_dict[x])

    fig = px.bar(df, x="sentiment_code", y="count", color ="sentiment_bin", barmode="group")
    #fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:

from search_or_simulate import SearchOrSimulate
from api_twitter import StreammerTwitter


streammer = SearchOrSimulate()
#streammer.simulate_search_streaming("tweets_search_2.txt", True, model_file = "best_keras_network_model.h5", cvectorizer_file = "count_vectorizer.joblib")
#streammer.search_streaming("streaming_tweets_search.txt", False, model_file = "best_keras_network_model.h5", cvectorizer_file = "count_vectorizer.joblib")
streammer.search_30_days_ago()
#transformed_tweet



ModuleNotFoundError: No module named 'search_or_simulate'